# UPDATE

### `MATCH ASSET_ID`

In [ ]:

UPDATE `olivier_it` it
LEFT JOIN `assets_filtree_reduced` aa
ON aa.asset_id = it.asset_id
and it.asset_id != '0'
SET it.status_asset_id = ( CASE WHEN aa.asset_id is not null then 'Trouvé - bon asset id'
                          ELSE "Non Trouvé - Mauvais Asset ID"
                          END 
                          )


### `MATCH POSITION`

In [ ]:

UPDATE `olivier_it` it 
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3` it2
ON it.reperage_id = it2.reperage_id

SET it.status_position = 'Bonne localisation'
WHERE  (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

In [ ]:
UPDATE `olivier_it` it 
SET it.status_position = 'KO localisation'
where it.status_position is null

# `Nom`

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON aa.nom_court  = it.short_name    
SET it.status_name = "OK nom_court"
where it.short_name != ''

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON aa.nom_long  = it.nom   
SET it.status_name = "OK nom_long"
where it.nom != '' and status_name is null

### netapp & ATTO & Brocade

In [ ]:
-- 42 rows affected
UPDATE   `olivier_it` it
 LEFT JOIN `it_built_system_NAS_v3` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status_2 = ( 
    			CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  	WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0' or it.asset_id ='') then  Concat(it.fabricant, ' : No Match') 
        		END )
                
WHERE it.status_2 is null
AND (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%')

### Exadata

In [ ]:
3 rows affected
UPDATE   `olivier_it` it
 LEFT JOIN `it_build_system_EXADATA_v3` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status_2 = ( 
    			CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  	WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END )
                
WHERE it.status_2 is NULL 
AND (it.fabricant like '%oracle%')

### HPC

#### No match

In [ ]:
0 rows affected
UPDATE `olivier_it` it
 LEFT JOIN `it_build_system_HPC_v3` it2
 ON (LEFT(it.Nom,6)) = it2.root_ferme

SET it.status_2 = ( 
    			CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.root_ferme) 
        	  	WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END )

WHERE (it.nom like '%HPC%' 
OR it.short_name like '%HPC%') and it.status_2 is NULL

### HANNA

In [ ]:
0 ROWS
UPDATE `olivier_it` it
  LEFT JOIN `it_build_system_HANNA_v3` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status_2 =(CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
				WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END ) 

WHERE it.status_2 is null and (it.nom_de_baie like '%hana%' OR it.nom like '%hana%')

### Ring

In [ ]:
0 rows
UPDATE `olivier_it` it
  LEFT JOIN `it_build_system_RING_v3` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status_2 =(CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
				WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END ) 

WHERE it.status_2 is NULL and (it.short_name like '%NA9TCR%' 
OR it.nom like '%NA9TCR%')

### PCP : no match

In [ ]:
0 rows
UPDATE `olivier_it` it
SET it.status_2 = 'PCP : No Match'
where (it.Nom like '%_Rack%' or it.short_name like '%_Rack%') and (asset_id is null or asset_id ='0') and status_2 is null

### FEX

In [ ]:
13 rows
UPDATE `olivier_it` it
LEFT JOIN `IT_Equipment_FEX_v3` it2 on it2.root_fex = (LEFT(it.Nom,8)) and it.Nom != it2.root_fex

SET it.status_2 =(CASE WHEN it.status_2 is null and it2.status_2 is not null and it2.asset_id !='0' then Concat(it.fabricant, ' : ', it2.asset_id) 
                WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it2.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
                END ) 

where it.Fabricant like '%cisco%' and it2.root_fex is not null and it.status_2 is null

### `'CMDB-OK-OLD'`

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name )
AND aa.asset_id = it.asset_id


SET it.status_name = (CASE WHEN nom_court like '%OLD' and  nom !='' THEN "nom_court type OLD"
                      WHEN nom_long like '%OLD' and  short_name !='' THEN "nom_long type OLD"
                      ELSE it.status_name
                      END
                     )

where (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD')

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name


SET it.status_name = (CASE WHEN nom_court like '%OLD' and  nom !='' THEN "nom_court type OLD"
                      WHEN nom_long like '%OLD' and  short_name !='' THEN "nom_long type OLD"
                      ELSE it.status_name
                      END
                     )

where (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD') and status_asset_id != 'Trouvé - bon asset id'

In [ ]:
UPDATE `olivier_it` it
SET it.status_name = "Mauvais nom"
where it.status_name is null

# `Status : Etat`

In [ ]:
UPDATE `olivier_it` it
LEFT JOIN `assets_filtree_reduced` aa
ON aa.asset_id = it.asset_id
and it.asset_id != '0'
SET it.status_etat = aa.etat


In [ ]:

UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name )

SET it.status_etat = aa.etat
where status_name = "nom_court type OLD"

In [ ]:

UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name )
SET it.status_etat = aa.etat
where status_name = "nom_long type OLD"

In [ ]:

UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name )
AND aa.asset_id = it.asset_id

SET it.status_etat = aa.etat
where status_name = "nom_court type OLD" and status_asset_id != 'Trouvé - bon asset id'

In [ ]:

UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name )
AND aa.asset_id = it.asset_id
SET it.status_etat = aa.etat
where status_name = "nom_long type OLD" and status_asset_id != 'Trouvé - bon asset id'

In [ ]:
UPDATE `olivier_it` it
 SET it.status_etat = 'KO Etat_CMDB'
where it.status_etat is null

# `GLOBAL STATUS`

In [ ]:
UPDATE olivier_it 
set status_global = concat(status_asset_id , ' - ' , status_position , ' - ' , status_name, " - " , status_etat )